In [114]:
import yfinance as yf
import pandas as pd
import numpy as np
import scipy.stats as sc
import scipy.linalg
import matplotlib.pyplot as plt
%matplotlib inline

In [115]:
sentiment = pd.read_csv("sentiment1.csv")
sentiment.head()

TotalSentimentScore  Vaccine  Death  Shut        Date
0             0.085862        0      0     0  02/11/2020
1            -0.098021        2      3     1  02/12/2020
2            -0.093167        0      3     0  02/13/2020
3             0.009190        0      1     2  02/14/2020
4             0.000000        1      0     0  02/15/2020

In [116]:
def get_data(stock_name):
    stock = yf.Ticker(stock_name).history(start="2020-02-11", end="2020-10-31") # from 2020-02-11 to 2020-10-30
    dates = []
    for x in range(len(stock)):
        newdate = str(stock.index[x])
        newdate = newdate[0:10]
        dates.append(newdate)
    stock['Date'] = dates
    
    split = []
    i = 0
    for date in sentiment.Date:
        if date in stock.Date:
            split.append(stock.iat[i,6])  # Stock Splits
            i+=1
        else:
            split.append(0)
    
    price = np.zeros(263)
    i = 0
    for date in sentiment.Date:
        if date in stock.Date:
            d = date[6:] + "-" + date[0:2] + "-" + date[3:5]  
            price[i] = stock[stock.Date == d].Close[0]
        else:
            price[i] = 0
        i+=1
        
    for i in range(1, len(price)):
        if price[i] == 0 :
            if i != len(price)-1:
                next_price = 0
                for j in range(i+1,i+7):
                    if price[j] != 0:
                        next_price = price[j]
                        break
                price[i] = 0.5 * (price[i-1] + next_price)
            else:
                price[i] = price[i-1]
    
    df = sentiment
    df["Price"] = price
    df["Stock Splits"] = split
    return df

In [117]:
zoom = get_data('ZM')
pd.DataFrame(zoom).to_csv(r'C:\Users\suziy\OneDrive\Desktop\ZM_adjclose.csv', index=False)
zoom

TotalSentimentScore  Vaccine  Death  Shut        Date       Price  \
0               0.085862        0      0     0  02/11/2020   88.680000   
1              -0.098021        2      3     1  02/12/2020   88.279999   
2              -0.093167        0      3     0  02/13/2020   89.860001   
3               0.009190        0      1     2  02/14/2020   90.949997   
4               0.000000        1      0     0  02/15/2020   93.669998   
..                   ...      ...    ...   ...         ...         ...   
258            -0.045096        6      4     3  10/26/2020  517.789978   
259            -0.109511        5      8     2  10/27/2020  538.989990   
260            -0.030883        4      4     8  10/28/2020  516.010010   
261            -0.052484        3      1     6  10/29/2020  489.679993   
262             0.143944        1      0     0  10/30/2020  460.910004   

     Stock Splits  
0               0  
1               0  
2               0  
3               0  
4               0  
..            ...  
258             0  
259             0  
260             0  
261             0  
262             0  

[263 rows x 7 columns]